# Importing libraries

In [61]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.preprocessing import LabelEncoder

# Read  the Dataset

In [2]:
data = pd.read_csv("C:\Excelr assignments\Assignment 13 Random Forest\Fraud_check (1).csv")

In [3]:
data.head(10)

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
5,NO,Divorced,33329,116382,0,NO
6,NO,Divorced,83357,80890,8,YES
7,YES,Single,62774,131253,3,YES
8,NO,Single,83519,102481,12,YES
9,YES,Divorced,98152,155482,4,YES


# EDA on data

In [19]:
data['Target'] = data['Taxable.Income'].apply(lambda x : 'Risky' if x >= 30000 else 'Good' )

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Undergrad        600 non-null    object
 1   Marital.Status   600 non-null    object
 2   Taxable.Income   600 non-null    int64 
 3   City.Population  600 non-null    int64 
 4   Work.Experience  600 non-null    int64 
 5   Urban            600 non-null    object
 6   Target           600 non-null    object
dtypes: int64(3), object(4)
memory usage: 32.9+ KB


In [21]:
data.describe

<bound method NDFrame.describe of     Undergrad Marital.Status  Taxable.Income  City.Population  \
0          NO         Single           68833            50047   
1         YES       Divorced           33700           134075   
2          NO        Married           36925           160205   
3         YES         Single           50190           193264   
4          NO        Married           81002            27533   
..        ...            ...             ...              ...   
595       YES       Divorced           76340            39492   
596       YES       Divorced           69967            55369   
597        NO       Divorced           47334           154058   
598       YES        Married           98592           180083   
599        NO       Divorced           96519           158137   

     Work.Experience Urban Target  
0                 10   YES  Risky  
1                 18   YES  Risky  
2                 30   YES  Risky  
3                 15   YES  Risky  
4    

In [22]:
data.shape

(600, 7)

In [23]:
data.duplicated().shape

(600,)

In [24]:
data2 = pd.get_dummies(data,columns=['Undergrad','Marital.Status','Urban']) 

In [25]:
data2

,Taxable.Income,City.Population,Work.Experience,Target,Undergrad_NO,Undergrad_YES,Marital.Status_Divorced,Marital.Status_Married,Marital.Status_Single,Urban_NO,Urban_YES
0,68833,50047,10,Risky,1,0,0,0,1,0,1
1,33700,134075,18,Risky,0,1,1,0,0,0,1
2,36925,160205,30,Risky,1,0,0,1,0,0,1
3,50190,193264,15,Risky,0,1,0,0,1,0,1
4,81002,27533,28,Risky,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
595,76340,39492,7,Risky,0,1,1,0,0,0,1
596,69967,55369,2,Risky,0,1,1,0,0,0,1
597,47334,154058,0,Risky,1,0,1,0,0,0,1
598,98592,180083,17,Risky,0,1,0,1,0,1,0


In [27]:
label = LabelEncoder()

In [28]:
data2['Target'] = label.fit_transform(data2['Target'])

In [29]:
data2

,Taxable.Income,City.Population,Work.Experience,Target,Undergrad_NO,Undergrad_YES,Marital.Status_Divorced,Marital.Status_Married,Marital.Status_Single,Urban_NO,Urban_YES
0,68833,50047,10,1,1,0,0,0,1,0,1
1,33700,134075,18,1,0,1,1,0,0,0,1
2,36925,160205,30,1,1,0,0,1,0,0,1
3,50190,193264,15,1,0,1,0,0,1,0,1
4,81002,27533,28,1,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
595,76340,39492,7,1,0,1,1,0,0,0,1
596,69967,55369,2,1,0,1,1,0,0,0,1
597,47334,154058,0,1,1,0,1,0,0,0,1
598,98592,180083,17,1,0,1,0,1,0,1,0


# Checking for outliers in the data

In [30]:
from sklearn.ensemble import IsolationForest

In [31]:
#training the model
clf = IsolationForest(random_state=20,contamination=0.01)
clf.fit(data2)

IsolationForest(contamination=0.01, random_state=20)

In [32]:
#predictions
y_pred_outliers = clf.predict(data2)

In [34]:
y_pred_outliers

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
       -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1

In [44]:
for x in y_pred_outliers:
    print(y_pred_outliers[x == -1].shape)

(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(1, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(1, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(0, 600)
(

# Defining X and Y variable 

In [50]:
array = data2.values 
len(array)

600

In [55]:
x = array[:,[0,1,2,4,5,6,7,8,9,10]]

In [56]:
y = array[:,3]

# Feature importance using Decision Tree

In [45]:
model = tree.DecisionTreeClassifier()

In [59]:
model.fit(x,y)

DecisionTreeClassifier()

In [60]:
print(model.feature_importances_)

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


# Model building 

In [63]:
kfold = KFold(n_splits=20)
model1 = RandomForestClassifier(n_estimators=100,max_features=3)
results = cross_val_score(model1,x,y,cv=kfold)

# Accuracy of model

In [64]:
print(results.mean())

0.9983333333333334
